In [2]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd
import numba
from numba import njit
import datetime
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as spop
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime as daee
from itertools import combinations
import json
import requests


In [22]:
datadirectory="C:/Users/jeanc/Documents/DatAAAAsss/Data/open.csv"

In [14]:
end = datetime.datetime.now()
start = end - datetime.timedelta(days=365*10)
vbt.settings.set_theme("dark")

In [24]:
data = pd.read_csv(datadirectory, parse_dates=True)
display(data)

,timestamp,open_CSCO,open_ISRG,open_TPR,open_DVN,open_MRO,open_BA,open_GILD,open_MDT,open_V,...,open_CI,open_BALL,open_JNJ,open_DOV,open_CRM,open_PGR,open_WAT,open_BWA,open_BLK,open_PPL
0,2023-10-06,52.99,286.47,27.41,43.00,24.36,185.370,74.46,75.60,233.17,...,285.92,47.57,157.08,137.69,200.48,142.80,263.97,38.51,635.02,22.68
1,2023-10-05,53.40,291.85,27.88,43.00,24.04,186.160,74.67,76.38,230.41,...,285.30,49.05,156.27,138.44,202.00,142.30,268.17,38.97,633.65,22.88
2,2023-10-04,53.66,289.62,27.90,44.94,25.23,188.850,73.30,76.92,229.48,...,284.35,49.24,155.57,137.61,200.88,139.41,265.97,38.73,629.03,22.65
3,2023-10-03,53.86,291.54,28.33,45.52,25.32,188.625,73.21,77.28,230.47,...,286.11,48.64,154.68,137.17,202.75,140.00,268.81,39.50,637.73,22.39
4,2023-10-02,53.84,291.96,28.62,47.82,26.79,191.470,75.00,77.90,229.24,...,285.71,49.74,155.42,139.04,200.80,138.89,273.36,40.29,644.71,23.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6017,1999-11-05,71.69,NaN,NaN,38.94,27.75,44.500,47.13,38.00,NaN,...,81.00,39.63,106.00,43.75,NaN,95.00,46.88,40.44,15.00,26.81
6018,1999-11-04,72.88,NaN,NaN,38.94,27.87,43.130,49.81,38.00,NaN,...,78.81,40.19,105.00,44.00,NaN,92.63,47.00,40.44,14.88,26.81
6019,1999-11-03,74.00,NaN,NaN,39.13,28.62,42.500,50.00,36.38,NaN,...,79.00,40.50,104.00,42.81,NaN,92.75,47.50,40.69,14.69,27.00
6020,1999-11-02,73.50,NaN,NaN,38.94,29.56,43.000,40.13,36.63,NaN,...,80.38,40.31,105.10,42.06,NaN,92.25,50.56,40.25,14.75,27.06


In [99]:

close_price=data["open_GILD"]

In [100]:
def ma_strategy(close_price,window,lower_multiple,upper_multiple):
    signal=np.full(close_price.shape[0],np.nan)
    for x in range(len(close_price)):
        if x >=window:
            ma=np.mean(close_price[x-window:x])
            if close_price[x]<ma*lower_multiple:
                signal[x]=1
            elif close_price[x]>ma*upper_multiple:
                signal[x]=-1
            else:
                signal[x]=0
    return signal

In [119]:
indicator= vbt.IndicatorFactory(
    class_name="Clubbed",
    short_name="Clu",
    input_names=["close_price"],
    param_names=["window","lower_multiple","upper_multiple"],
    output_names=["signal"],
).with_apply_func(
    ma_strategy,
    window=40,
    lower_multiple=2,
    upper_multiple=10,)

In [120]:
results = indicator.run(close_price)
results.signal

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
6017    1.0
6018    1.0
6019    1.0
6020    1.0
6021    1.0
Name: open_GILD, Length: 6022, dtype: float64

In [121]:
entries= results.signal ==1
exits =results.signal  ==-1

In [122]:
portfolio=vbt.Portfolio.from_signals(close_price,entries,exits,fees=0.001,init_cash=100000) 


In [123]:
portfolio.stats()

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\base\wrapping.py:971: UserWarning:

Couldn't parse the frequency of index. Pass it as `freq` or define it globally under `settings.wrapping`.



Start                                     0
End                                    6021
Period                                 6022
Start Value                        100000.0
Min Value                      33059.749407
Max Value                     152223.988999
End Value                      78610.118421
Total Return [%]                 -21.389882
Benchmark Return [%]             -15.055063
Total Time Exposure [%]           99.335769
Max Gross Exposure [%]                100.0
Max Drawdown [%]                  78.282169
Max Drawdown Duration                3937.0
Total Orders                              1
Total Fees Paid                     99.9001
Total Trades                              1
Win Rate [%]                            NaN
Best Trade [%]                          NaN
Worst Trade [%]                         NaN
Avg Winning Trade [%]                   NaN
Avg Losing Trade [%]                    NaN
Avg Winning Trade Duration              NaN
Avg Losing Trade Duration       

In [124]:
portfolio.value.vbt.plot()

FigureWidget({
    'data': [{'name': 'open_GILD',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'd4a9077f-9b12-4e67-a7cc-46eda00a6336',
              'x': array([   0,    1,    2, ..., 6019, 6020, 6021], dtype=int64),
              'y': array([100000.        , 100000.        , 100000.        , ...,  62142.38610357,
                           49875.47908673,  78610.11842102])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})